## MAGIC Gamma Telescope


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Dataset:

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Donated by:
P. Savicky
Institute of Computer Science, AS of CR
Czech Republic
savicky '@' cs.cas.cz


In [2]:
df = pd.read_csv('magic04.data', header=None)
df.columns = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1',
              'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [3]:
df['class'] = df['class'].map({'g': 1, 'h': 0})
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


In [4]:
train, validate, test = np.split(
    df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

/home/animesh/miniconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
# !conda install -c conda-forge imbalanced-learn --yes

In [6]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [7]:
def scale_data(data, overSample=False):
    X = data.drop('class', axis=1)
    y = data['class']

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    if overSample:
        ros = RandomOverSampler()
        X, y = ros.fit_resample(X, y)

    data = np.hstack((X, np.array(y).reshape(-1, 1)))

    return data, X, y

In [8]:
print(len(train[train['class'] == 1]), len(train[train['class'] == 0]))

7441 3971


In [9]:
train, X_train, y_train = scale_data(train, overSample=True)

In [10]:
print(len(train[train[:, -1] == 1]), len(train[train[:, -1] == 0]))

7441 7441


In [11]:
validate, X_validate, y_validate = scale_data(validate)
test, X_test, y_test = scale_data(test)

## k-nearest neighbors


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [13]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [14]:
y_pred = knn_model.predict(X_test)

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.68      0.72      1354
           1       0.83      0.89      0.86      2450

    accuracy                           0.81      3804
   macro avg       0.80      0.78      0.79      3804
weighted avg       0.81      0.81      0.81      3804



## Naive Bayes

The Naive Bayes formula is:

$ P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)} $

Here are the names for each term:

- $ P(A|B) $: **Posterior Probability** - the probability of event $A$ occurring given that event $B$ has occurred.
- $ P(B|A) $: **Likelihood** - the probability of event $B$ occurring given that event $A$ has occurred.
- $ P(A) $: **Prior Probability** - the probability of event $A$ occurring independently of event $B$.
- $ P(B) $: **Marginal Probability** or **Evidence** - the probability of event $B$ occurring independently of event $A$.

In the context of Naive Bayes classification:

- $ A $: The class label.
- $ B $: The feature vector or evidence.

So, in terms of classification:

- $ P(A|B) $: The probability of the class $A$ given the feature vector $B$.
- $ P(B|A) $: The probability of the feature vector $B$ given the class $A$.
- $ P(A) $: The prior probability of the class $A$.
- $ P(B) $: The prior probability of the feature vector $B$.


In [16]:
from sklearn.naive_bayes import GaussianNB

In [17]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

In [18]:
y_pred_nb = nb_model.predict(X_test)
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

           0       0.68      0.38      0.49      1354
           1       0.73      0.90      0.80      2450

    accuracy                           0.72      3804
   macro avg       0.70      0.64      0.65      3804
weighted avg       0.71      0.72      0.69      3804



## Logistic Regression

- Let the independent input features be:

$ X = \begin{bmatrix}
x*{11} & \cdots & x*{1m} \\
x*{21} & \cdots & x*{2m} \\
\vdots & \ddots & \vdots \\
x*{n1} & \cdots & x*{nm}
\end{bmatrix} $

and the dependent variable is Y having only binary value i.e. 0 or 1.

$
Y = \begin{cases} 
0 & \text{if } \textit{Class 1} \\
1 & \text{if } \textit{Class 2} 
\end{cases} 
$

- then, apply the multi-linear function to the input variables X.

$ z = \left( \sum\_{i=1}^{n} w_i x_i \right) + b $<br>
$z=w.X+b$

- Sigmoid Function
  Now we use the sigmoid function where the input will be z and we find the probability between 0 and 1. i.e. predicted y.

$ \sigma(z) = \frac{1}{1+e^{-z}} $

- logistic regression equation

$ y = \sigma(w.X+b) $


In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
lg_model = LogisticRegression()
lg_model.fit(X_train, y_train)

LogisticRegression()

In [21]:
y_pred_lg = lg_model.predict(X_test)
print(classification_report(y_test, y_pred_lg))

              precision    recall  f1-score   support

           0       0.70      0.73      0.71      1354
           1       0.85      0.82      0.83      2450

    accuracy                           0.79      3804
   macro avg       0.77      0.78      0.77      3804
weighted avg       0.79      0.79      0.79      3804



## svm


In [22]:
from sklearn.svm import SVC

In [23]:
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [24]:
y_pred_svm = svm_model.predict(X_test)
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.82      0.80      0.81      1354
           1       0.89      0.90      0.90      2450

    accuracy                           0.87      3804
   macro avg       0.85      0.85      0.85      3804
weighted avg       0.87      0.87      0.87      3804



## Neural Network


In [25]:
import tensorflow as tf

2024-06-19 09:39:58.760251: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [38]:
def plot_history(history):
    fig,(ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))
    ax1.plot(history.history['loss'], label='loss')
    ax1.plot(history.history['val_loss'], label='val_loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Binary Crossentropy')
    ax1.grid(True)

    ax2.plot(history.history['accuracy'], label='accuracy')
    ax2.plot(history.history['val_accuracy'], label='val_accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax1.grid(True)

    plt.show()

In [32]:
def train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
    nn_model = tf.keras.Sequential([
        tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(10,)),
        tf.keras.layers.Dropout(dropout_prob),
        tf.keras.layers.Dense(num_nodes, activation='relu'),
        tf.keras.layers.Dropout(dropout_prob),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                     loss='binary_crossentropy', metrics=['accuracy'])

    history = nn_model.fit(X_train, y_train, epochs=epochs,
                           batch_size=batch_size, validation_split=0.2, verbose=0)

    return nn_model, history

In [ ]:

epochs = 100
least_val_loss = float('inf')
least_loss_model = None
for num_nodes in [16, 32, 64]:
    for dropout_prob in [0, 0.2]:
        for lr in [0.01, 0.005, 0.001]:
            for batch_size in [32, 64, 128]:
                model, history = train_model(
                    X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs)
                print(f'num_nodes: {num_nodes}, dropout_prob: {dropout_prob}, lr: {lr}, batch_size: {batch_size}')
                plot_history(history)
                val_loss = model.evaluate(X_validate, y_validate)[0]
                if val_loss < least_val_loss:
                    least_val_loss = val_loss
                    least_loss_model = model

In [43]:
y_pred_nn=least_loss_model.predict(X_test)
y_pred_nn = (y_pred_nn > 0.5).astype(int).reshape(-1)

119/119 [==============================] - 1s 5ms/step


In [44]:
print(classification_report(y_test, y_pred_nn))

              precision    recall  f1-score   support

           0       0.88      0.76      0.82      1354
           1       0.88      0.94      0.91      2450

    accuracy                           0.88      3804
   macro avg       0.88      0.85      0.86      3804
weighted avg       0.88      0.88      0.88      3804

